In [1]:
# Import libraries
import markov_clustering as mcl
import networkx as nx
import pandas as pd
from collections import Counter
from tqdm import tqdm
import pyarrow as pa
import pyarrow.parquet as pq

from networkx.algorithms.community import greedy_modularity_communities
from networkx import edge_betweenness_centrality as betweenness
from networkx.algorithms.community.centrality import girvan_newman
import itertools

In [2]:
df5 = pd.read_parquet('df5_latest_v5.parquet',engine='pyarrow')

In [7]:
df5

,Style-Fabric-Color (code)_x,Style-Fabric-Color (code)_y,Jaccard,mc_cluster_x,cluster_size_x,mc_cluster_y,cluster_size_y
0,A79448-A225588-A7008,A81722-A229950-A7205,0.021739,3764,5,109,12
1,AUD01039-A232741-A1008,A81722-A229950-A7205,0.002646,2204,5,109,12
2,AUD01039-A232741-A1008,A81722-A229950-A7205,0.002646,2204,5,109,12
3,AU10026-AC00059-A80G,A81722-A229950-A7205,0.002058,3790,3,109,12
4,V800683S-VJ00366-V7008,A81722-A229950-A7205,0.009524,435,3,109,12
...,...,...,...,...,...,...,...
27861811,EB2HWA729-E11620-E003,EE0YWASF6-E71960-EM53,0.166667,3270,2,3270,2
27861812,1000424-1A01388-5P030,1000205-1A00328-2B150,0.250000,5047,2,5047,2
27861813,N25814-N403651-N1211,ZPL14001-ZWSM0003-Z4001,0.166667,5699,2,5699,2
27861814,A74192-A216622-A1363,A72158-A213311-A1001,0.125000,4828,2,4828,2


In [ ]:
count=0

while df5['mc_cluster_x'].nunique() > 15 and count <10:
            
    #Repeat groupby to reflect changes to clusters
    df6=df5.groupby(by= ['mc_cluster_x', 'mc_cluster_y',
                         'cluster_size_x','cluster_size_y']).agg({"Jaccard" : "mean"}).reset_index().sort_values(by=['cluster_size_x','Jaccard','cluster_size_y'], ascending=[True,False,True])
    
    if df6['mc_cluster_x'].iloc[0]!=df6['mc_cluster_y'].iloc[0]:
        
        #New cluster ID - changing the cluster ID of x in df5 to the top mc_cluster_y in df6
        df5.loc[df5['mc_cluster_x']==df6['mc_cluster_x'].iloc[0],'mc_cluster_x']=df6['mc_cluster_y'].iloc[0]
        
        #Reassigning cluster_size_x in df5 - as we're merging it with the top entry for cluster_size_y in df6   
        df5.loc[df5['mc_cluster_x']==df6['mc_cluster_y'].iloc[0],'cluster_size_x']=df6['cluster_size_x'].iloc[0]+df6['cluster_size_y'].iloc[0]
        
        #Replace corresponding cluster_size_y in df5 where we've changed the size
        df5.loc[df5['mc_cluster_y']==df6['mc_cluster_y'].iloc[0],'cluster_size_y']=df6['cluster_size_x'].iloc[0]+df6['cluster_size_y'].iloc[0]
        
        count+=1

In [4]:
#see number of clusters we get
print(df5['mc_cluster_x'].nunique())

4742


In [ ]:
#Export the dataframe after the loop for the fifth time and read
pq.write_table(pa.Table.from_pandas(df5),'df5_latest_v6.parquet')
df5 = pd.read_parquet('df5_latest_v6.parquet',engine='pyarrow')